
# 🔳 QR Code Art with Diffusion + ControlNet (Colab Ready)

***Jungwoo Ahn***

**Workflow:** Select model → Set resolution → Input link (+ optional image) → Generate QR code → ControlNet → Output stylized, scannable QR art.

**Included model combos (pick 1 at runtime):**
1. **SDXL (High Quality)**: `stabilityai/stable-diffusion-xl-base-1.0` + **Canny ControlNet (SDXL)** — great detail and fidelity.
2. **SD1.5 + Canny (Reliable & Simple)**: `runwayml/stable-diffusion-v1-5` + `lllyasviel/sd-controlnet-canny` — classic pipeline.
3. **SD1.5 + QRCode-Monster (Specialized)**: `runwayml/stable-diffusion-v1-5` + `monster-labs/control_v1p_sd15_qrcode_monster` — designed to preserve QR readability while stylizing.

> The notebook supports **ControlNet conditioning** from the generated QR (edges or specialized QR ControlNet). You can also optionally feed a **style image** (image-to-image) to influence appearance while keeping the QR scannable.



> **Tip (Colab):** Go to **Runtime → Change runtime type → GPU** for big speed-ups.


In [ ]:

#@title ⬇️ Install dependencies
!pip -q install --upgrade pip
!pip -q install "diffusers>=0.29.0" transformers accelerate safetensors xformers
!pip -q install opencv-python pillow qrcode[pil] numpy matplotlib ipywidgets


In [ ]:

#@title 📦 Imports
import os, io, math, time, random
import numpy as np
import torch
from PIL import Image
import cv2
import qrcode
import matplotlib.pyplot as plt
from typing import Optional, Tuple

from diffusers import (
    StableDiffusionXLControlNetPipeline,
    StableDiffusionControlNetPipeline,
    ControlNetModel,
    AutoencoderKL,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
)

from diffusers.utils import load_image

def show(img, title=None):
    plt.figure(figsize=(6,6))
    plt.axis('off')
    if title: plt.title(title)
    plt.imshow(img if isinstance(img, np.ndarray) else np.array(img))
    plt.show()

def ensure_even(x:int)->int:
    return int(x//8*8)  # most models like dims divisible by 8

def to_pil(img: Image.Image | np.ndarray) -> Image.Image:
    if isinstance(img, Image.Image):
        return img.convert("RGB")
    return Image.fromarray(img).convert("RGB")

def make_qr(link:str, size:int=768, border:int=4) -> Image.Image:
    qr = qrcode.QRCode(
        version=None,  # automatic
        error_correction=qrcode.constants.ERROR_CORRECT_H,  # robust after stylizing
        box_size=10,
        border=border,
    )
    qr.add_data(link)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")
    img = img.resize((size, size), Image.NEAREST)
    return img

def canny_edges(img: Image.Image, low: int=100, high: int=200) -> Image.Image:
    arr = np.array(img.convert("L"))
    edges = cv2.Canny(arr, low, high)
    edges_rgb = np.stack([edges]*3, axis=-1)
    return Image.fromarray(edges_rgb)

def overlay_qr(alpha_img: Image.Image, qr_mask: Image.Image, alpha: float=0.25) -> Image.Image:
    """Optional: softly mix the raw QR onto the generated result to boost scan reliability."""
    a = np.array(alpha_img.convert("RGB"), dtype=np.float32) / 255.0
    q = np.array(qr_mask.convert("RGB"), dtype=np.float32) / 255.0
    # In QR, black modules are 0; we softly darken corresponding pixels
    mixed = np.clip(a*(1.0) - (1.0-q)*alpha, 0, 1)
    return Image.fromarray((mixed*255).astype(np.uint8))

def save_image(img: Image.Image, path: str) -> str:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    img.save(path)
    return path


In [ ]:

#@title ⚙️ Settings (edit these and run)
#@markdown **Choose a model combo**
MODEL_COMBO = "SDXL + Canny (SDXL)"  #@param ["SDXL + Canny (SDXL)", "SD1.5 + Canny", "SD1.5 + QRCode-Monster"]

#@markdown **Resolution**
RESOLUTION = "768"  #@param ["512", "768", "1024"]
RESOLUTION = int(RESOLUTION)

#@markdown **Your link (encoded into QR)**
# LINK = "https://example.com"  #@param {type:"string"}
LINK = "https://github.com/jungwooahn721"  #@param {type:"string"}

#@markdown **Optional style image URL or upload path (leave empty if none)**
STYLE_IMAGE_URL_OR_PATH = ""  #@param {type:"string"}

#@markdown **Prompt / Negative prompt**
PROMPT = "a vibrant, futuristic cyberpunk poster, neon glow, high detail, symmetrical balance"  #@param {type:"string"}
NEGATIVE_PROMPT = "blurry, low quality, distorted, extra text, artifacts"  #@param {type:"string"}

#@markdown **Sampler steps & strength (if using style init image)**
NUM_STEPS = 25  #@param {type:"slider", min:1, max:50, step:1}
GUIDANCE_SCALE = 5.0  #@param {type:"slider", min:0.0, max:15.0, step:0.5}
INIT_IMAGE_STRENGTH = 0.35  #@param {type:"slider", min:0.0, max:1.0, step:0.05}

#@markdown **Canny thresholds (if using canny control)**
CANNY_LOW = 100  #@param {type:"slider", min:0, max:255, step:1}
CANNY_HIGH = 200  #@param {type:"slider", min:0, max:255, step:1}

#@markdown **Seed (set -1 for random)**
SEED = -1  #@param {type:"number"}

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


In [ ]:

#@title 🧠 Load model(s)
import torch

torch_dtype = torch.float16 if (device=="cuda") else torch.float32

if MODEL_COMBO == "SDXL + Canny (SDXL)":
    base_model = "stabilityai/stable-diffusion-xl-base-1.0"
    controlnet_id = "diffusers/controlnet-canny-sdxl-1.0"  # SDXL ControlNet (canny)
    controlnet = ControlNetModel.from_pretrained(controlnet_id, torch_dtype=torch_dtype)
    pipe = StableDiffusionXLControlNetPipeline.from_pretrained(
        base_model,
        controlnet=controlnet,
        torch_dtype=torch_dtype,
        variant="fp16" if torch_dtype==torch.float16 else None,
        use_safetensors=True,
    )
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    if device == "cpu":
        pipe.enable_model_cpu_offload()
    else:
        pipe.to(device)

elif MODEL_COMBO == "SD1.5 + Canny":
    base_model = "runwayml/stable-diffusion-v1-5"
    controlnet_id = "lllyasviel/sd-controlnet-canny"
    controlnet = ControlNetModel.from_pretrained(controlnet_id, torch_dtype=torch_dtype)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        base_model,
        controlnet=controlnet,
        torch_dtype=torch_dtype,
        safety_checker=None,
        feature_extractor=None,
        use_safetensors=True,
    )
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception as _:
        pass
    if device == "cpu":
        pipe.enable_model_cpu_offload()
    else:
        pipe.to(device)

elif MODEL_COMBO == "SD1.5 + QRCode-Monster":
    base_model = "runwayml/stable-diffusion-v1-5"
    controlnet_id = "monster-labs/control_v1p_sd15_qrcode_monster"  # specialized QR ControlNet
    controlnet = ControlNetModel.from_pretrained(controlnet_id, torch_dtype=torch_dtype)
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        base_model,
        controlnet=controlnet,
        torch_dtype=torch_dtype,
        safety_checker=None,
        feature_extractor=None,
        use_safetensors=True,
    )
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception as _:
        pass
    if device == "cpu":
        pipe.enable_model_cpu_offload()
    else:
        pipe.to(device)

else:
    raise ValueError("Unknown MODEL_COMBO selection.")

print("Loaded:", MODEL_COMBO)


In [ ]:

#@title 🧩 Generate QR (and optional style image)
W = H = ensure_even(RESOLUTION)

qr_img = make_qr(LINK, size=W, border=4)
show(qr_img, "Raw QR (high error correction)")

style_img = None
if STYLE_IMAGE_URL_OR_PATH.strip():
    try:
        if STYLE_IMAGE_URL_OR_PATH.startswith("http"):
            style_img = load_image(STYLE_IMAGE_URL_OR_PATH)
        else:
            style_img = Image.open(STYLE_IMAGE_URL_OR_PATH)
        style_img = style_img.convert("RGB").resize((W, H))
        show(style_img, "Style image (resized)")
    except Exception as e:
        print("Failed to load style image:", e)
        style_img = None

# Control image (Canny or QR-specialized)
if MODEL_COMBO in ["SDXL + Canny (SDXL)", "SD1.5 + Canny"]:
    control_img = canny_edges(qr_img, CANNY_LOW, CANNY_HIGH)
    show(control_img, "ControlNet condition (Canny edges from QR)")
else:
    # QRCode-Monster expects the raw QR silhouette as control (binary-ish)
    control_img = qr_img
    show(control_img, "ControlNet condition (raw QR)")


In [ ]:

#@title 🎨 Generate QR code art
g_seed = SEED if SEED >= 0 else random.randint(0, 2**32-1)
generator = torch.Generator(device=device).manual_seed(g_seed)
print("Seed:", g_seed)

common_kwargs = dict(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    num_inference_steps=NUM_STEPS,
    guidance_scale=GUIDANCE_SCALE,
    generator=generator,
)

if MODEL_COMBO.startswith("SDXL"):
    # SDXL ControlNet expects image/control_image; optional 'image' for img2img via 'strength'
    if style_img is not None:
        out = pipe(
            **common_kwargs,
            image=style_img,           # SDXL img2img support
            strength=INIT_IMAGE_STRENGTH,
            control_image=control_img.resize((W,H)),
            width=W,
            height=H,
        )
    else:
        out = pipe(
            **common_kwargs,
            control_image=control_img.resize((W,H)),
            width=W,
            height=H,
        )
else:
    # SD1.5 ControlNet pipeline (txt2img or img2img-like with init_image)
    if style_img is not None:
        out = pipe(
            **common_kwargs,
            image=style_img,
            control_image=control_img.resize((W,H)),
            strength=INIT_IMAGE_STRENGTH,
        )
    else:
        out = pipe(
            **common_kwargs,
            control_image=control_img.resize((W,H)),
            width=W,
            height=H,
        )

gen = out.images[0]

# Optional: softly re-impose the QR modules for maximum scan reliability
final_img = overlay_qr(gen, qr_img, alpha=0.20)

show(gen, "Generated (pre-overlay)")
show(final_img, "Final (soft QR overlay for reliability)")

save_dir = "/content" if os.path.isdir("/content") else "/mnt/data"
raw_path = save_image(gen, os.path.join(save_dir, f"qr_art_raw_{g_seed}.png"))
final_path = save_image(final_img, os.path.join(save_dir, f"qr_art_final_{g_seed}.png"))
qr_path = save_image(qr_img, os.path.join(save_dir, f"qr_raw_{g_seed}.png"))
print("Saved:")
print(" -", qr_path)
print(" -", raw_path)
print(" -", final_path)



## ✅ Tips for Scannability
- Keep **error correction H** (already set) for robustness.
- Prefer **high contrast** between QR modules and background after stylizing.
- Use **Canny thresholds** that preserve square modules (100/200 is a good start).
- If scans fail, **increase overlay alpha** (in code) or reduce `INIT_IMAGE_STRENGTH` (if using style image).

## Prompting Hints
- Keep compositions **balanced/symmetric**; avoid heavy warping.
- Add keywords like *"clean edges, crisp geometry, high contrast, centered, minimal clutter"*.
- For SDXL, quality tags like *"photorealistic, sharp details, 8k"* can help, but don't overdo it.
